<a href="https://colab.research.google.com/github/thejawker/architector/blob/master/Architector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Is this going to be working?

does it work to save it to GH?